In [1]:
import psycopg2
import csv
import pandas as pd
import plotly.express as px

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    database="osm",
    user="postgres",
    password="postgres")

cursor = conn.cursor()
cursor.execute("""
    SELECT poi.poi_id, poi.name, poi_coordinates.latitude, poi_coordinates.longitude, poi.district_id
    FROM poi 
    JOIN poi_coordinates ON poi.poi_id=poi_coordinates.poi_id""")

data = cursor.fetchall()

cursor.close()
conn.close()

In [ ]:
# with open('D:\\vkrb\\csv\\all_points.csv', 'w', newline='', encoding='utf-8-sig') as file:
#     record = csv.writer(file)
#     record.writerow(['poi_id', 'name', 'latitude', 'longitude', 'district_id'])
#     record.writerows(data)

In [5]:
# бесплатный ключ доступа можно найти здесь https://account.mapbox.com
MAPBOX_ACCESS_TOKEN = "pk.eyJ1IjoiY3NpY3NhY3NvIiwiYSI6ImNsaWFpM3B2bzAzcTUzbXFwZ2ZjdnVpajEifQ.UY-B4Tg9KH0NXNC423X7Jg"

In [9]:
data = pd.read_csv('D:\\vkrb\\csv\\all_points.csv')

In [10]:
# подсчет точек в каждом district_id
district_sum =data.groupby('district_id').size().reset_index(name='Плотность')
# слияние данных с исходным ДФ для добавления количества точек в каждом районе
data_district_sum = pd.merge(data, district_sum, on='district_id')

px.set_mapbox_access_token(MAPBOX_ACCESS_TOKEN)
fig = px.scatter_mapbox(data_district_sum, lat='latitude', lon='longitude',
                        color='Плотность', size='Плотность',
                        width=1000, height=800, 
                        hover_name='name', # hover_data='district_id',
                        mapbox_style='open-street-map')
fig.show()